<a href="https://colab.research.google.com/github/Aakash-Modi/AI-Training/blob/main/tree/main/voice-analytics/assignments/CEN-2021-Odd/Team-7/conformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Conformer (ESPnet)

In [ ]:
# mount google drive to access data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# installing required dependencies
!pip install datasets
!pip install librosa
!pip install -q espnet==0.10.0
!pip install -q espnet_model_zoo
!pip install jiwer
!pip install pytorch-model-summary

     |████████████████████████████████| 290 kB 5.3 MB/s 
     |████████████████████████████████| 132 kB 48.2 MB/s 
     |████████████████████████████████| 243 kB 41.8 MB/s 
     |████████████████████████████████| 1.1 MB 29.7 MB/s 
     |████████████████████████████████| 59 kB 5.7 MB/s 
     |████████████████████████████████| 192 kB 45.0 MB/s 
     |████████████████████████████████| 271 kB 32.0 MB/s 
     |████████████████████████████████| 160 kB 47.6 MB/s 
     |████████████████████████████████| 787 kB 5.5 MB/s 
     |████████████████████████████████| 1.7 MB 33.3 MB/s 
     |████████████████████████████████| 1.5 MB 33.1 MB/s 
     |████████████████████████████████| 1.0 MB 39.9 MB/s 
     |████████████████████████████████| 13.1 MB 134 kB/s 
     |████████████████████████████████| 212 kB 43.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 174 kB 42.6 MB/s 
     |██

In [ ]:
# import required packages
import json
import librosa
import time
import torch
import string
from espnet_model_zoo.downloader import ModelDownloader
from espnet2.bin.asr_inference import Speech2Text
import pandas as pd
import soundfile
import librosa.display
from IPython.display import display, Audio
import matplotlib.pyplot as plt
import datasets
import os

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.


In [ ]:
# load test clean librispeech dataset (created using https://colab.research.google.com/drive/1jYEUzVAeh089VTcCGxLErTka0sZ7-GuH?usp=sharing)
!unzip '/content/drive/MyDrive/Colab Notebooks/Speech_Processing/test_clean_data/file.zip'
with open("/content/drive/MyDrive/Colab Notebooks/Speech_Processing/test_clean_data/test_clean.json") as json_file:
    test_dict = json.load(json_file)
test_dict

Archive:  /content/drive/MyDrive/Colab Notebooks/Speech_Processing/test_clean_data/file.zip
   creating: content/test_clean_data/
  inflating: content/test_clean_data/61-70970-0033.flac  
  inflating: content/test_clean_data/121-127105-0025.flac  
  inflating: content/test_clean_data/7176-92135-0023.flac  
  inflating: content/test_clean_data/8230-279154-0009.flac  
  inflating: content/test_clean_data/6829-68769-0014.flac  
  inflating: content/test_clean_data/1089-134686-0004.flac  
  inflating: content/test_clean_data/4446-2273-0034.flac  
  inflating: content/test_clean_data/1580-141083-0022.flac  
  inflating: content/test_clean_data/4446-2273-0007.flac  
  inflating: content/test_clean_data/6930-81414-0015.flac  
  inflating: content/test_clean_data/7021-79759-0001.flac  
  inflating: content/test_clean_data/5639-40744-0032.flac  
  inflating: content/test_clean_data/2830-3980-0029.flac  
  inflating: content/test_clean_data/2830-3980-0040.flac  
  inflating: content/test_clean_d

{'0': {'length': 10.435,
  'text': 'HE HOPED THERE WOULD BE STEW FOR DINNER TURNIPS AND CARROTS AND BRUISED POTATOES AND FAT MUTTON PIECES TO BE LADLED OUT IN THICK PEPPERED FLOUR FATTENED SAUCE',
  'wav': '{data_root}/1089-134686-0000.flac'},
 '1': {'length': 3.275,
  'text': 'STUFF IT INTO YOU HIS BELLY COUNSELLED HIM',
  'wav': '{data_root}/1089-134686-0001.flac'},
 '2': {'length': 6.625,
  'text': 'AFTER EARLY NIGHTFALL THE YELLOW LAMPS WOULD LIGHT UP HERE AND THERE THE SQUALID QUARTER OF THE BROTHELS',
  'wav': '{data_root}/1089-134686-0002.flac'},
 '3': {'length': 2.68,
  'text': 'HELLO BERTIE ANY GOOD IN YOUR MIND',
  'wav': '{data_root}/1089-134686-0003.flac'},
 '4': {'length': 5.2150625,
  'text': 'NUMBER TEN FRESH NELLY IS WAITING ON YOU GOOD NIGHT HUSBAND',
  'wav': '{data_root}/1089-134686-0004.flac'},
 '5': {'length': 9.635,
  'text': "THE MUSIC CAME NEARER AND HE RECALLED THE WORDS THE WORDS OF SHELLEY'S FRAGMENT UPON THE MOON WANDERING COMPANIONLESS PALE FOR WEARINESS",


In [ ]:
# parameters
# model: https://github.com/espnet/espnet_model_zoo/blob/master/espnet_model_zoo/table.csv 
lang = 'en'
fs = 16000
tag = 'kamo-naoyuki/librispeech_asr_train_asr_conformer5_raw_bpe5000_scheduler_confwarmup_steps25000_batch_bins140000000_optim_conflr0.0015_initnone_accum_grad2_sp_valid.acc.ave'
#tag = 'kamo-naoyuki/librispeech_asr_train_asr_conformer5_raw_bpe5000_frontend_confn_fft400_frontend_confhop_length160_scheduler_confwarmup_steps25000_batch_bins140000000_optim_conflr0.0015_initnone_sp_valid.acc.ave'
#tag = 'kamo-naoyuki/librispeech_asr_train_asr_conformer5_raw_bpe5000_frontend_confn_fft512_frontend_confhop_length256_scheduler_confwarmup_steps25000_batch_bins140000000_optim_conflr0.0015_initnone_sp_valid.acc.ave'
#tag = 'kamo-naoyuki/librispeech_asr_train_asr_conformer6_n_fft512_hop_length256_raw_en_bpe5000_scheduler_confwarmup_steps40000_optim_conflr0.0025_sp_valid.acc.ave'
#tag = 'byan/librispeech_asr_train_asr_conformer_raw_bpe_batch_bins30000000_accum_grad3_optim_conflr0.001_sp'

In [ ]:
d = ModelDownloader()
# model
speech2text = Speech2Text(
    **d.download_and_unpack(tag),
    device="cuda",
    minlenratio=0.0,
    maxlenratio=0.0,
    ctc_weight=0.3,
    beam_size=10,
    batch_size=0,
    nbest=1
)

# text normalization
def text_normalizer(text):
    text = text.upper()
    return text.translate(str.maketrans('', '', string.punctuation))

https://zenodo.org/record/4541452/files/asr_train_asr_conformer5_raw_bpe5000_scheduler_confwarmup_steps25000_batch_bins140000000_optim_conflr0.0015_initnone_accum_grad2_sp_valid.acc.ave.zip?download=1: 100%|██████████| 649M/649M [00:30<00:00, 22.4MB/s]


In [ ]:
speech2text.asr_model.parameters

<bound method Module.parameters of ESPnetASRModel(
  (frontend): DefaultFrontend(
    (stft): Stft(n_fft=512, win_length=512, hop_length=128, center=True, normalized=False, onesided=True)
    (frontend): Frontend()
    (logmel): LogMel(sr=16000, n_fft=512, n_mels=80, fmin=0, fmax=8000.0, htk=False)
  )
  (specaug): SpecAug(
    (time_warp): TimeWarp(window=5, mode=bicubic)
    (freq_mask): MaskAlongAxis(mask_width_range=[0, 30], num_mask=2, axis=freq)
    (time_mask): MaskAlongAxis(mask_width_range=[0, 40], num_mask=2, axis=time)
  )
  (normalize): GlobalMVN(stats_file=/usr/local/lib/python3.7/dist-packages/espnet_model_zoo/f48f96312da93d298fda581eb0b11c2b/exp/asr_stats_raw_bpe5000_sp/train/feats_stats.npz, norm_means=True, norm_vars=True)
  (encoder): ConformerEncoder(
    (embed): Conv2dSubsampling(
      (conv): Sequential(
        (0): Conv2d(1, 512, kernel_size=(3, 3), stride=(2, 2))
        (1): ReLU()
        (2): Conv2d(512, 512, kernel_size=(3, 3), stride=(2, 2))
        (3): 

In [ ]:
pytorch_total_params = sum(p.numel() for p in speech2text.asr_model.parameters())
pytorch_total_params

116146960

In [ ]:
# ASR
predicted=[]
actual=[]
for i in range(1301, len(test_dict)):
  path = os.path.join("/content/content/test_clean_data/"+test_dict[str(i)]["wav"].split("/")[-1])
  speech, sr= soundfile.read(path)
  nbests = speech2text(speech)
  
  text, *_ = nbests[0]
  if (i%50==0):
    print(f"Input Speech: {i}")
  # let us listen to samples
  predicted.append(text_normalizer(text))
  actual.append(text_normalizer(test_dict[str(i)]["text"]))

/usr/local/lib/python3.7/dist-packages/torch/functional.py:573: UserWarning: stft will soon require the return_complex parameter be given for real inputs, and will further require that return_complex=True in a future PyTorch release. (Triggered internally at  ../aten/src/ATen/native/SpectralOps.cpp:659.)
  normalized, onesided, return_complex)
/usr/local/lib/python3.7/dist-packages/espnet2/layers/stft.py:109: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  olens = (ilens - self.win_length) // self.hop_length + 1
/usr/local/lib/python3.7/dist-packages/espnet/nets/ctc_prefix_score.py:326: RuntimeWarning: invalid value encountered in logaddexp
  r_prev[:, 0], r_pre

Input Speech: 1350
Input Speech: 1400
Input Speech: 1450
Input Speech: 1500
Input Speech: 1550
Input Speech: 1600
Input Speech: 1650
Input Speech: 1700
Input Speech: 1750
Input Speech: 1800
Input Speech: 1850
Input Speech: 1900
Input Speech: 1950
Input Speech: 2000
Input Speech: 2050
Input Speech: 2100
Input Speech: 2150
Input Speech: 2200
Input Speech: 2250
Input Speech: 2300
Input Speech: 2350
Input Speech: 2400
Input Speech: 2450
Input Speech: 2500
Input Speech: 2550
Input Speech: 2600


In [ ]:
# wer metric
metric = datasets.load_metric('wer')
wer_score = metric.compute(predictions=predicted, references=actual)
print(wer_score)

Downloading:   0%|          | 0.00/1.95k [00:00<?, ?B/s]

0.026115172602525508


In [ ]:
# save list as .txt file
f = open("test_clean_2.txt", "w")
json.dump(predicted, f)
f.close()

In [ ]:
f = open("test_clean_2act.txt", "w")
json.dump(actual, f)
f.close()

In [ ]:
f = open("/content/drive/MyDrive/Colab Notebooks/test_clean_1.txt", "r")
p_1 = json.load(f)
f.close()

In [ ]:
f = open("/content/drive/MyDrive/Colab Notebooks/test_clean_2.txt", "r")
p_2 = json.load(f)
f.close()

In [ ]:
p_1.extend(p_2)
len(p_1)

2620

In [ ]:
f = open("/content/drive/MyDrive/Colab Notebooks/test_clean_1act.txt", "r")
a_1 = json.load(f)
f.close()

In [ ]:
f = open("/content/drive/MyDrive/Colab Notebooks/test_clean_2act.txt", "r")
a_2 = json.load(f)
f.close()

In [ ]:
a_1.extend(a_2)
len(a_1)

2620

In [ ]:
wer_score = metric.compute(predictions=p_1, references=a_1)
print(wer_score)

0.025753195374315276
